### Accessing different climate weather data file through AWS S3 Bucket.
  - grib
  - netcdf
  - csv

Reference:https://climatedataguide.ucar.edu/climate-tools/common-climate-data-formats-overview

### DataSets on AWS in this notebook:
  - Global Historical Climatology Network (GHCN) https://registry.opendata.aws/noaa-ghcn/
  - GOES-16 & GOES 17  & GOES 18 https://registry.opendata.aws/noaa-goes/
  - Climate Forecast System (CFS) https://registry.opendata.aws/noaa-cfs/
  - Global Forecast System (GFS) https://registry.opendata.aws/noaa-gfs-bdp-pds/
  - NREL National Solar Radiation Database (NSRDB) https://registry.opendata.aws/nrel-pds-nsrdb/
  
Whenever people mention dataset on AWS, retrieve the bucket name and put it after the following URL:
https://s3.console.aws.amazon.com/s3/buckets/<br> 
Take this notebook for example, if we are interested in the "noaa-gfs-bdp-pds" bucket,<br> the URL wiil be: https://s3.console.aws.amazon.com/s3/buckets/noaa-gfs-bdp-pds<br> Then you can see the objects details in the bucket, especially S3 URI. 

## .grib2  .grb2  .grib file

This is the only way I found to read grib file in S3 Bucket using engine `cfgrib` under `xarray`.......

Reference Documentation: https://stackoverflow.com/questions/66229140/xarray-read-remote-grib-file-on-s3-using-cfgrib

In [1]:
import xarray as xr
import fsspec
import json
import s3fs 

# add "simplecache::" to the S3 URI for grib file
GFS_Remote_Grib_S3_URI = "simplecache::s3://noaa-gfs-bdp-pds/sst.20220627/rtgssthr_grb_0.083.grib2"
GFS_My_Grib_S3_URI = "simplecache::s3://justindemo123/rtgssthr_grb_0.083.grib2"
CFS_Grib_S3_URI = "simplecache::s3://noaa-cfs-pds/cdas.20240308/cdas1.t00z.sfluxgrbl02.grib2"

In [2]:
Remote_file = fsspec.open_local(GFS_Remote_Grib_S3_URI, 
                         s3 = {'anon' : True}, 
                         filecache = {'cache_storage':'/tmp/files'})

GFS_Remote_Grib_ds = xr.open_dataset(Remote_file, engine="cfgrib")

GFS_Remote_Grib_ds

<xarray.Dataset>
Dimensions:     (latitude: 2160, longitude: 4320)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.07 -89.16 -89.96
  * longitude   (longitude) float64 0.042 0.1253 0.2087 ... 359.8 359.9 360.0
    valid_time  datetime64[ns] ...
Data variables:
    t           (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-03-12T09:42 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
# need access key and secret access key to access data stored in your own S3 Bucket
# Retrieve credentials from a local file
with open('credentials.json') as f:
     creds = json.load(f)

My_file = fsspec.open_local(GFS_My_Grib_S3_URI, 
                            s3 = {"key": creds['key'], 
                                  "secret" : creds['secret']}, 
                            filecache = {'cache_storage':'/tmp/files'})

GFS_My_Grib_ds = xr.open_dataset(My_file, engine="cfgrib")

GFS_My_Grib_ds

<xarray.Dataset>
Dimensions:     (latitude: 2160, longitude: 4320)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.07 -89.16 -89.96
  * longitude   (longitude) float64 0.042 0.1253 0.2087 ... 359.8 359.9 360.0
    valid_time  datetime64[ns] ...
Data variables:
    t           (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-03-12T09:42 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
CFS_file = fsspec.open_local(CFS_Grib_S3_URI, 
                         s3 = {'anon' : True}, 
                         filecache = {'cache_storage':'/tmp/files'})

CFS_Grib_ds = xr.open_dataset(CFS_file, filter_by_keys={'stepType': 'avg', 'typeOfLevel': 'surface'}, engine="cfgrib")

CFS_Grib_ds

<xarray.Dataset>
Dimensions:     (latitude: 94, longitude: 192)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 88.54 86.65 84.75 ... -84.75 -86.65 -88.54
  * longitude   (longitude) float64 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
    valid_time  datetime64[ns] ...
Data variables: (12/27)
    utaua       (latitude, longitude) float32 ...
    vtaua       (latitude, longitude) float32 ...
    msshf       (latitude, longitude) float32 ...
    mslhf       (latitude, longitude) float32 ...
    dlwrf       (latitude, longitude) float32 ...
    ulwrf       (latitude, longitude) float32 ...
    ...          ...
    snowc       (latitude, longitude) float32 ...
    csdlf       (latitude, longitude) float32 ...
    csusf       (latitude, longitude) float32 ...
    csdsf       (latitude, longitude) float32 ...
    csulf       (latitude, longitude) float32 ...
    snohf       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-03-12T09:42 GRIB to CDM+CF via cfgrib-0.9.1...

## .nc .h5 file 

In [5]:
GFS_NC_S3_URI = "s3://noaa-gfs-bdp-pds/enkfgdas.20210325/12/atmos/mem073/gdas.t12z.sfcf003.nc"
NSRDB_H5_S3_URI = "s3://nrel-pds-nsrdb/philippines/philippines_2017.h5"
GOES_NC_S3_URI = 's3://noaa-goes18/SEIS-L1b-EHIS/2024/003/02/OR_SEIS-L1b-EHIS_G18_s20240030204360_e20240030209350_c20240030210374.nc'

In [6]:
fs = s3fs.S3FileSystem(anon=True)

with fs.open(GFS_NC_S3_URI) as fileObj:
    GFS_nc_ds = xr.open_dataset(fileObj, engine='h5netcdf')

GFS_nc_ds

<xarray.Dataset>
Dimensions:        (grid_xt: 1536, grid_yt: 768, time: 1)
Coordinates:
  * grid_xt        (grid_xt) float64 0.0 0.2344 0.4688 ... 359.3 359.5 359.8
  * grid_yt        (grid_yt) float64 89.82 89.59 89.35 ... -89.35 -89.59 -89.82
  * time           (time) object 2021-03-25 15:00:00
Data variables: (12/150)
    lon            (grid_yt, grid_xt) float64 ...
    lat            (grid_yt, grid_xt) float64 ...
    acond          (time, grid_yt, grid_xt) float32 ...
    albdo_ave      (time, grid_yt, grid_xt) float32 ...
    alnsf          (time, grid_yt, grid_xt) float32 ...
    alnwf          (time, grid_yt, grid_xt) float32 ...
    ...             ...
    xtts           (time, grid_yt, grid_xt) float32 ...
    xu             (time, grid_yt, grid_xt) float32 ...
    xv             (time, grid_yt, grid_xt) float32 ...
    xz             (time, grid_yt, grid_xt) float32 ...
    xzts           (time, grid_yt, grid_xt) float32 ...
    zc             (time, grid_yt, grid_xt) float32 ...
Attributes:
    fhzero:       6
    ncld:         5
    nsoil:        4
    imp_physics:  11
    dtp:          240.0
    source:       FV3GFS
    grid:         gaussian
    im:           1536
    jm:           768

In [7]:
with fs.open(NSRDB_H5_S3_URI) as fileObj:
    NSRDB_h5_ds = xr.open_dataset(fileObj, backend_kwargs={"phony_dims": "sort"}, engine='h5netcdf')

NSRDB_h5_ds

<xarray.Dataset>
Dimensions:          (phony_dim_0: 8760, phony_dim_1: 31353, phony_dim_2: 2)
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_2
Data variables:
    air_temperature  (phony_dim_0, phony_dim_1) float32 ...
    coordinates      (phony_dim_1, phony_dim_2) float32 ...
    dhi              (phony_dim_0, phony_dim_1) float32 ...
    dni              (phony_dim_0, phony_dim_1) float32 ...
    ghi              (phony_dim_0, phony_dim_1) float32 ...
    meta             (phony_dim_1) [('site', 'S22'), ('country', 'S11'), ('longitude', '<f4'), ('latitude', '<f4'), ('elevation', '<f4'), ('timezone', '<i2'), ('dataset', 'S13')] ...
    time_index       (phony_dim_0) |S25 ...
    wind_speed       (phony_dim_0, phony_dim_1) float32 ...

In [8]:
with fs.open(GOES_NC_S3_URI) as fileObj:
    goes_nc_ds = xr.open_dataset(fileObj, engine='h5netcdf')

goes_nc_ds

<xarray.Dataset>
Dimensions:                                      (report_number: 1, energy: 5,
                                                  energy_bounds: 2,
                                                  error_bounds: 2, element: 26,
                                                  minute_interval: 5,
                                                  number_of_time_bounds: 2,
                                                  solar_array_current_channel_index: 4)
Dimensions without coordinates: report_number, energy, energy_bounds,
                                error_bounds, element, minute_interval,
                                number_of_time_bounds,
                                solar_array_current_channel_index
Data variables: (12/64)
    H5MinuteDifferentialFluxes                   (report_number, energy) float32 ...
    H5MinuteDifferentialFluxStatErrors           (report_number, energy) float32 ...
    H5MinuteDifferentialFluxInstErrors           (report_number, energy) float32 ...
    H5MinuteDifferentialEnergyBounds             (report_number, energy, energy_bounds) float32 ...
    He5MinuteDifferentialFluxes                  (report_number, energy) float32 ...
    He5MinuteDifferentialFluxStatErrors          (report_number, energy) float32 ...
    ...                                           ...
    element_label                                (element) |S2 ...
    error_bounds_label                           (error_bounds) |S10 ...
    solar_array_current_channel_index_label      (solar_array_current_channel_index) |S25 ...
    minute_interval_label                        (minute_interval) |S8 ...
    percent_uncorrectable_L0_errors              float32 ...
    algorithm_dynamic_input_data_container       int32 ...
Attributes: (12/29)
    production_site:              WCDAS
    dataset_name:                 OR_SEIS-L1b-EHIS_G18_s20240030204360_e20240...
    naming_authority:             gov.nesdis.noaa
    Conventions:                  CF-1.7
    institution:                  DOC/NOAA/NESDIS> U.S. Department of Commerc...
    project:                      GOES
    ...                           ...
    date_created:                 2024-01-03T02:10:37.4Z
    L1b_processing_parm_version:  OR_SEISS-L1b-PARM-EHIS_G18_v01r00.zip
    algorithm_version:            OR_SEIS-L1b-ALG-EHIS_v01r00.zip
    product_version:              v01r00
    LUT_Filenames:                SeissEhisCalInrParameters(FM3A_CDRL88RevB_D...
    id:                           97bb330a-ef6e-48d3-8679-49101c8ae190

## .csv file

In [9]:
GHCN_csv_S3_URI = "s3://noaa-ghcn-pds/csv/by_year/2024.csv"

In [10]:
import pandas as pd

GHCN_csv_S3_data = pd.read_csv(GHCN_csv_S3_URI)
GHCN_csv_S3_data

,ID,DATE,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
0,AE000041196,20240101,TMAX,278,NaN,NaN,S,NaN
1,AE000041196,20240101,TMIN,182,NaN,NaN,S,NaN
2,AE000041196,20240101,PRCP,0,D,NaN,S,NaN
3,AE000041196,20240101,TAVG,236,H,NaN,S,NaN
4,AEM00041194,20240101,TMAX,277,NaN,NaN,S,NaN
...,...,...,...,...,...,...,...,...
5543848,VQ1VIST0013,20240310,PRCP,0,NaN,NaN,N,700.0
5543849,VQ1VIST0013,20240310,SNOW,0,NaN,NaN,N,700.0
5543850,VQC00672560,20240310,PRCP,0,NaN,NaN,H,700.0
5543851,WQW00041606,20240310,PRCP,0,NaN,NaN,S,NaN
